<a href="https://colab.research.google.com/github/gvelec/AppliedDataScience/blob/master/PGA_Segment%26Clust_Toronto_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto


> In the next cells I propose a code in python according with the instruccions for this peer-graded assignement.

First I need to load the libraries required to scrap the data.

In [1]:
# WEB SCRAPING
# Table from Wikipedia
#### Finding Postal Codes, Borough, and Neighbourhood
#
# loading the libraries required
from bs4 import BeautifulSoup
import requests
import pandas as pd
#

In this point I need to prepare the parameters to make the request data process.

In [2]:
# variables to handle the request
query = {'table':''}
url = ' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # web to scrap
#

With this code I going to get the data from the response and extract the items required from the data.

In [3]:
# make the request
response = requests.get(url, params=query)
#
# get the full data from the response
data = response.text
soup = BeautifulSoup(data, 'html.parser')
table = soup.find_all('table')[0] # Grab the original table
#
# to know the original table rows number
row_marker = 0
for row in table.find_all('tr'):
    row_marker += 1
#
# building a new table to be filled with the correct data
new_table = pd.DataFrame(columns=['Postal Code','Borough','Neighbourhood'],index=range(0,row_marker)) # Now I know the size and the columns names
#
# getting the items requested  
row_marker = 0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        new_table.iat[row_marker,column_marker] = column.get_text()[:-1]
        column_marker += 1
    row_marker += 1
new_table = new_table.drop([0])
#

In this code I did the data cleanning and fix the table according with the instructions.

In [4]:
# eliminating the rows with blank items
new_table.dropna(inplace=True)
#
# filtering the Borough from Toronto
toronto_boroughs=['North York','East York','York','Scarborough','Etobicoke','Downtown Toronto','East Toronto','West Toronto','North Toronto','Central Toronto'] # according with the Toronto City Map
new_table_in = new_table[new_table.Borough.isin(toronto_boroughs)]    # Valid Postal Codes
new_table_not = new_table[~new_table.Borough.isin(toronto_boroughs)]  # Not valid Postal Codes
#
# eliminating the rows with Borough == 'Not assigned'
new_table_in.drop(new_table_in[new_table_in.Borough == 'Not assigned'].index, inplace=True)    # Valid Postal Codes
new_table_not.drop(new_table_not[new_table_not.Borough == 'Not assigned'].index, inplace=True) # Not valid Postal Codes
#
# assigning the Borough like Neighbourhood for Neighbourhood == 'Not assigned'
new_table_in['Neighbourhood'].mask(new_table_in['Neighbourhood'] == 'Not assigned', new_table_in['Borough'], inplace=True)    # Valid Postal Codes
new_table_not['Neighbourhood'].mask(new_table_not['Neighbourhood'] == 'Not assigned', new_table_not['Borough'], inplace=True) # Not valid Postal Codes
#
# reseting the indexes
new_table_in = new_table_in.reset_index(drop=True)    # Valid Postal Codes
new_table_not = new_table_not.reset_index(drop=True)  # Not valid Postal Codes
#
# searching for Neighbourhoods with multiple Valid Postal Codes
multiple_postal_codes = new_table_in[new_table_in.duplicated(subset=['Neighbourhood'],keep=False)]
df_multiple = pd.DataFrame(multiple_postal_codes)
df_multiple_group = df_multiple.groupby(['Neighbourhood'])['Postal Code'].apply(','.join).reset_index()
multiple_postal_neighbourhoods = df_multiple_group['Neighbourhood'].size # Neighbourhoods with multiple valid postal codes
#
# counting multiple valid postal codes
multiple_code = df_multiple['Neighbourhood'].value_counts(ascending=True)
#

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


If was a sussecful process, now I can see the final results with some info about it.

In [5]:
# output some logs for monitoring
#
# some stats
valid = new_table_in.to_numpy().shape[0]
not_valid = new_table_not.to_numpy().shape[0]
total_scraped = valid+not_valid
#
print('Scrap process complete ==>\n')
print('==================================================================================================')
print('    Valid Postal Codes ==> {} for Area of Toronto City According with the Map of the city.\n'.format(not_valid))
print('Not Valid Postal Codes ==> {} for Area of Toronto City According with the Map of the city.'.format(valid))
print('==================================================================================================\n')
print('This Neighbourghoods have a Not Valid Postal Code to Toronto City Area')
print('{:.2%} ==> {} Not Valid Postal Codes Scraped for "{}" : {}.\n'.format((not_valid/total_scraped),not_valid,new_table_not['Neighbourhood'][0],new_table_not['Postal Code'][0]))
print('This Neighbourghoods have a Valid Postal Code to Toronto City Area (Multiple Valid Codes included)')
print('{:.2%} ==> {} Valid Postal Codes Scraped.\n'.format((valid/total_scraped),valid))
print('This Neighbourghoods have Multiple Valid Postal Codes to Toronto City Area')
marker = 0
for count in multiple_code:
  print('{:.2%} ==> {} Valid Postal Codes Scraped for "{}" : {}.'.format((count/total_scraped),count,df_multiple_group['Neighbourhood'][marker],df_multiple_group['Postal Code'][marker]))
  marker = marker+1
print('\n==================================================================================================\n')
#
# output the results
new_table_in

Scrap process complete ==>

    Valid Postal Codes ==> 1 for Area of Toronto City According with the Map of the city.

Not Valid Postal Codes ==> 102 for Area of Toronto City According with the Map of the city.

This Neighbourghoods have a Not Valid Postal Code to Toronto City Area
0.97% ==> 1 Not Valid Postal Codes Scraped for "Canada Post Gateway Processing Centre" : M7R.

This Neighbourghoods have a Valid Postal Code to Toronto City Area (Multiple Valid Codes included)
99.03% ==> 102 Valid Postal Codes Scraped.

This Neighbourghoods have Multiple Valid Postal Codes to Toronto City Area
1.94% ==> 2 Valid Postal Codes Scraped for "Don Mills" : M3B,M3C.
3.88% ==> 4 Valid Postal Codes Scraped for "Downsview" : M3K,M3L,M3M,M3N.




,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
97,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
98,M4Y,Downtown Toronto,Church and Wellesley
99,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
100,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


The next code gives me the possibility to save the results like a .csv file in my Google Drive, in order to be used for different Notebooks in the next task.

In [6]:
# getting access to my drive as local repository
from google.colab import drive
ROOT = "/content/drive/"
drive.mount(ROOT)
# convert results to a csv file and save them in the local repository
new_table_in.to_csv('results.csv', index=False)
!cp results.csv "drive/My Drive/DataScience"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
